In [1]:
using DiffEqSensitivity, OrdinaryDiffEq, Zygote

function fiip(du,u,p,t)
  du[1] = dx = p[1]*u[1] - p[2]*u[1]*u[2]
  du[2] = dy = -p[3]*u[2] + p[4]*u[1]*u[2]
end
p = [1.5,1.0,3.0,1.0]; 
u0 = [1.0;1.0]
prob = ODEProblem(fiip,u0,(0.0,10.0),p)
alg = Rosenbrock23(autodiff=false)
sol = concrete_solve(prob,alg,abstol=1e-10,reltol=1e-10)

t: 12217-element Array{Float64,1}:
  0.0
  7.500165102325e-7
  8.250181612557501e-6
  8.325183263580749e-5
  0.0002441493755446294
  0.00045178872549769665
  0.0007759054563281014
  0.0011453416720713576
  0.0016248493710973929
  0.002136896713287465
  0.0027338071839994793
  0.0033500968758512543
  0.004024826225756027
  ⋮
  9.990964054515306
  9.991823212781544
  9.992682371047781
  9.993541529314019
  9.994400687580256
  9.995259845846494
  9.996119004112732
  9.99697816237897
  9.997837320645207
  9.998696478911445
  9.999555637177682
 10.0
u: 12217-element Array{Array{Float64,1},1}:
 [1.0, 1.0]
 [1.000000375008888, 0.9999984999682452]
 [1.0000041251673797, 0.9999834997899213]
 [1.000041633713319, 0.999833511928259]
 [1.0001221417434631, 0.9995118353468834]
 [1.00022612396585, 0.9990968816663428]
 [1.0003886298938207, 0.9984495429631414]
 [1.0005741462547422, 0.9977122660138839]
 [1.0008153938134468, 0.9967562352700614]
 [1.0010735831766677, 0.9957364665239684]
 [1.0013753067367803

In [2]:
@time du01,dp1 = Zygote.gradient((u0,p)->concrete_solve(prob,alg,u0,p,save_everystep=false,save_start=false)[end,1],u0,p)

 38.706106 seconds (72.00 M allocations: 3.900 GiB, 4.11% gc time)


([-2.7943765147377064, 0.21125041458994295], [-6.324985452698097, -0.7048510776198609, -1.718940767730094, -2.78739184601153])

In [6]:
@time du01,dp1 = Zygote.gradient((u0,p)->concrete_solve(prob,Tsit5(),u0,p,save_everystep=false,save_start=false)[end,1],u0,p)

 17.446035 seconds (43.62 M allocations: 2.152 GiB, 5.68% gc time)


([-2.735337842312109, 0.21064211814849126], [-6.2414474188247135, -0.6972987072620919, -1.707491832271775, -2.7365828183600724])

In [3]:
sol_adj = solve(prob,alg,abstol=1e-10,reltol=1e-10)

function dg(out,u, p, t, i)
  out .= [1.0, 0.0]
end
ts = sol_adj.t[end]
@time res = adjoint_sensitivities(sol_adj,Rosenbrock23(autodiff=false),dg,ts,abstol=1e-9,reltol=1e-6)

  5.744803 seconds (16.77 M allocations: 720.386 MiB, 7.29% gc time)


([-1.965981873803132, -0.1885714562731049], [-2.160524417509413 -0.1885777736493886 -0.5631793149554023 -0.939630665744495])

In [4]:
u1 = copy(u0)
p1 = copy(p)
h = 0.001
u1[1] = u1[1] + h
# p1[1] = p1[1] + h
prob1 = ODEProblem(fiip,u1,(0.0,10.0),p1)
sol1 = concrete_solve(prob1,alg,abstol=1e-10,reltol=1e-10)

du1_fd = (sol1[end] - sol[end])[1]/h

1.968782939011593